In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
import string 
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from nltk.stem.wordnet import WordNetLemmatizer
import os

In [ ]:
def read_single_csv(input_path, use_cols):
    import pandas as pd
    df_chunk=pd.read_csv(input_path,chunksize=100000, error_bad_lines=False,usecols=use_cols)
    res_chunk=[]
    for chunk in df_chunk:
        res_chunk.append(chunk)
    res_df=pd.concat(res_chunk)
    return res_df

In [ ]:
use_cols = ['tweetcreatedts','text','language']
df = read_single_csv('data/data3.csv',use_cols)
df.head()

,tweetcreatedts,text,language
0,2022-04-01 00:00:00.000000,⚡The Ukrainian Air Force would like to address...,en
1,2022-04-01 00:00:00.000000,Chernihiv oblast. Ukrainians welcome their lib...,en
2,2022-04-01 00:00:00.000000,America 🇺🇸 is preparing for something worse th...,en
3,2022-04-01 00:00:00.000000,JUST IN: #Anonymous has hacked &amp; released ...,en
4,2022-04-01 00:00:00.000000,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...,en


In [ ]:
df

,tweetcreatedts,text,language
0,2022-04-01 00:00:00.000000,⚡The Ukrainian Air Force would like to address...,en
1,2022-04-01 00:00:00.000000,Chernihiv oblast. Ukrainians welcome their lib...,en
2,2022-04-01 00:00:00.000000,America 🇺🇸 is preparing for something worse th...,en
3,2022-04-01 00:00:00.000000,JUST IN: #Anonymous has hacked &amp; released ...,en
4,2022-04-01 00:00:00.000000,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...,en
...,...,...,...
27042952,2022-03-31 23:43:12.000000,"@Abiy Ahmed, Eritrean dictator Isaias Afewerki...",en
27042953,2022-03-31 23:43:12.000000,"#MARIUPOL—However, even at #Ilyich, #Ukraine|i...",en
27042954,2022-03-31 23:43:12.000000,🎥 دفعة جديدة من القوات الشيشانية انضمت للاعمال...,ar
27042955,2022-03-31 23:43:12.000000,🇺🇦 PRAY FOR UKRAINE K9\nペット先進国ウクライナ\nウクライナK-9達...,ja


In [ ]:
df2 = df[['tweetcreatedts','text']][df['language']=='en']
df2

,tweetcreatedts,text
0,2022-04-01 00:00:00.000000,⚡The Ukrainian Air Force would like to address...
1,2022-04-01 00:00:00.000000,Chernihiv oblast. Ukrainians welcome their lib...
2,2022-04-01 00:00:00.000000,America 🇺🇸 is preparing for something worse th...
3,2022-04-01 00:00:00.000000,JUST IN: #Anonymous has hacked &amp; released ...
4,2022-04-01 00:00:00.000000,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...
...,...,...
27042949,2022-03-31 23:43:11.000000,"Amb. William Taylor, former U.S. Ambassador to..."
27042951,2022-03-31 23:43:12.000000,@lapatina_ @berehulak this just hurts my heart...
27042952,2022-03-31 23:43:12.000000,"@Abiy Ahmed, Eritrean dictator Isaias Afewerki..."
27042953,2022-03-31 23:43:12.000000,"#MARIUPOL—However, even at #Ilyich, #Ukraine|i..."


In [ ]:
df2['tweetcreatedts'] = df2['tweetcreatedts'].apply(lambda x:x[:10])
df2

,tweetcreatedts,text
0,2022-04-01,⚡The Ukrainian Air Force would like to address...
1,2022-04-01,Chernihiv oblast. Ukrainians welcome their lib...
2,2022-04-01,America 🇺🇸 is preparing for something worse th...
3,2022-04-01,JUST IN: #Anonymous has hacked &amp; released ...
4,2022-04-01,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...
...,...,...
27042949,2022-03-31,"Amb. William Taylor, former U.S. Ambassador to..."
27042951,2022-03-31,@lapatina_ @berehulak this just hurts my heart...
27042952,2022-03-31,"@Abiy Ahmed, Eritrean dictator Isaias Afewerki..."
27042953,2022-03-31,"#MARIUPOL—However, even at #Ilyich, #Ukraine|i..."


In [ ]:
df2['tweetcreatedts'] = pd.to_datetime(df2['tweetcreatedts'])
df2.sort_values('tweetcreatedts', inplace=True)
df2

,tweetcreatedts,text
11736061,2022-02-24,BREAKING - JUST NOW: VP Kamala Harris gives a...
11667515,2022-02-24,#Ukraine 🇺🇦: anti-war protesters fill the stre...
11667514,2022-02-24,The Indiana House has unanimously supported it...
11667511,2022-02-24,"More ""corrupt billionaires"" of #Russia also be..."
11667510,2022-02-24,#FLASH—#Russia|n Ministry of Defence confirms ...
...,...,...
11222143,2022-05-01,🤗Residents of the #Kharkiv region liberated fr...
11222144,2022-05-01,#Russia delivered 2 tons of humanitarian aid i...
11222148,2022-05-01,"The ""Freedom of #Russia"" Legion reports that i..."
11222130,2022-05-01,All those missiles #Russia launched on #Ukrain...


In [ ]:
dates = df2['tweetcreatedts'].unique()
#print(len(dates))
periods = []
for p in range(11):
    tmp = []
    for i in range(6):
        tmp.append(dates[p*6+i])
    periods.append(tmp)

periods[-1].append(dates[-1])
periods

[[numpy.datetime64('2022-02-24T00:00:00.000000000'),
  numpy.datetime64('2022-02-25T00:00:00.000000000'),
  numpy.datetime64('2022-02-26T00:00:00.000000000'),
  numpy.datetime64('2022-02-27T00:00:00.000000000'),
  numpy.datetime64('2022-02-28T00:00:00.000000000'),
  numpy.datetime64('2022-03-01T00:00:00.000000000')],
 [numpy.datetime64('2022-03-02T00:00:00.000000000'),
  numpy.datetime64('2022-03-03T00:00:00.000000000'),
  numpy.datetime64('2022-03-04T00:00:00.000000000'),
  numpy.datetime64('2022-03-05T00:00:00.000000000'),
  numpy.datetime64('2022-03-06T00:00:00.000000000'),
  numpy.datetime64('2022-03-07T00:00:00.000000000')],
 [numpy.datetime64('2022-03-08T00:00:00.000000000'),
  numpy.datetime64('2022-03-09T00:00:00.000000000'),
  numpy.datetime64('2022-03-10T00:00:00.000000000'),
  numpy.datetime64('2022-03-11T00:00:00.000000000'),
  numpy.datetime64('2022-03-12T00:00:00.000000000'),
  numpy.datetime64('2022-03-13T00:00:00.000000000')],
 [numpy.datetime64('2022-03-14T00:00:00.000

In [ ]:
X_test = []
for p in periods:
    tmp = []
    for d in range(len(p)):
        tmp.append(df2[df2['tweetcreatedts']==p[d]])
    X_test.append(pd.concat(tmp))
X_test

[         tweetcreatedts                                               text
 11736061     2022-02-24  BREAKING -  JUST NOW: VP Kamala Harris gives a...
 11667515     2022-02-24  #Ukraine 🇺🇦: anti-war protesters fill the stre...
 11667514     2022-02-24  The Indiana House has unanimously supported it...
 11667511     2022-02-24  More "corrupt billionaires" of #Russia also be...
 11667510     2022-02-24  #FLASH—#Russia|n Ministry of Defence confirms ...
 ...                 ...                                                ...
 13676874     2022-03-01  @predsjednik_cg @USAmbMNE STOP RUSSIAN AGGRESS...
 13676883     2022-03-01  @MustafaSentop @RTErdogan @EmineErdogan STOP R...
 13676880     2022-03-01  It is unacceptable for #Russia that some Europ...
 13676879     2022-03-01  At this point I'm convinced I have no fhking i...
 13676882     2022-03-01  We’re Ready To Impose Sanctions On Russia For ...
 
 [1446030 rows x 2 columns],
          tweetcreatedts                                 

In [ ]:
len(X_test)

11

In [ ]:
for p in range(len(X_test)):
    X_test[p].to_csv("data\\raw\\raw_data"+str(p)+".csv", header='true', encoding='utf-8')

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt   



spec_chars = ["RT @[_A-Za-z0-9]+:","!",'"',"%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for p in X_test:
    for char in spec_chars:
        p['text'] = p['text'].str.replace(char, ' ')
        p['text'] = p['text'].str.lower()


    #df2['tweet'] = df2['tweet'].str.lower()
    p['text'] = np.vectorize(remove_pattern)(p['text'], "@[\w]*") 
    #df_test['tidy_Tweet'] = df_test['tidy_Tweet'].str.replace("[^a-zA-Z#]", " ")
    p['text'] = p['text'].str.replace("[^a-zA-Z]", " ")
    p['text'] = p['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
    p['text'] = p['text'].apply(lambda x: ' '.join([word.lower() for word in x.split() if word not in stop_words]))
    lmtzr = WordNetLemmatizer()
    p['text'] = p['text'].apply(lambda x: ' '.join([lmtzr.lemmatize(w,'v') for w in x.split()]))

X_test

<ipython-input-44-ea5678daaf6a>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  p['text'] = p['text'].str.replace(char, ' ')
<ipython-input-44-ea5678daaf6a>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  p['text'] = p['text'].str.replace(char, ' ')
<ipython-input-44-ea5678daaf6a>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  p['text'] = p['text'].str.replace("[^a-zA-Z]", " ")


MemoryError: Unable to allocate 6.09 GiB for an array with shape (1621933,) and data type <U1008

In [1]:
for p in X_test[7:]:
    print('done 1 period')
    for char in spec_chars:
        p['text'] = p['text'].str.replace(char, ' ')
        p['text'] = p['text'].str.lower()


    #df2['tweet'] = df2['tweet'].str.lower()
    p['text'] = np.vectorize(remove_pattern)(p['text'], "@[\w]*") 
    #df_test['tidy_Tweet'] = df_test['tidy_Tweet'].str.replace("[^a-zA-Z#]", " ")
    p['text'] = p['text'].str.replace("[^a-zA-Z]", " ")
    p['text'] = p['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
    p['text'] = p['text'].apply(lambda x: ' '.join([word.lower() for word in x.split() if word not in stop_words]))
    lmtzr = WordNetLemmatizer()
    p['text'] = p['text'].apply(lambda x: ' '.join([lmtzr.lemmatize(w,'v') for w in x.split()]))

NameError: ignored

In [ ]:
X_test[7:]

[        tweetcreatedts                                               text
 2738921     2022-04-07   anselm eduard71  cn nazi  fuheng h  titouusag...
 2738899     2022-04-07  🇷🇺❗️russia has been suspended from the un huma...
 2738927     2022-04-07  almost 90  of the murdered civilians in bucha ...
 2738900     2022-04-07  the #russian people need to remove  kremlinrus...
 2738901     2022-04-07  un general assembly votes to suspend russia s ...
 ...                ...                                                ...
 4548631     2022-04-12  ukraine s zelenskiy  calls on eu to sanction r...
 4548621     2022-04-12  a lucky girl now safe ❤️ more interviews  info...
 4548635     2022-04-12  according to the art of propaganda war that th...
 4548632     2022-04-12  ❗️rates of hunger in west africa are expected ...
 4548634     2022-04-12  as much as the world responded quickly for the...
 
 [1621933 rows x 2 columns],
         tweetcreatedts                                              

In [ ]:
for p in range(len(X_test)-4):
    print(X_test[p])

         tweetcreatedts                                               text
11736061     2022-02-24  break kamala harris give update russia ukraine...
11667515     2022-02-24  ukraine anti war protesters fill streets mosco...
11667514     2022-02-24  indiana house unanimously support sanction rus...
11667511     2022-02-24  corrupt billionaires russia also sanction acco...
11667510     2022-02-24  flash russia ministry defence confirm water fl...
...                 ...                                                ...
13676874     2022-03-01  predsjednik usambmne stop russian aggression u...
13676883     2022-03-01  mustafasentop rterdogan emineerdogan stop russ...
13676880     2022-03-01  unacceptable russia european countries host nu...
13676879     2022-03-01  point convince fhking idea wtf happen ukrainei...
13676882     2022-03-01  ready impose sanction russia invade ukraine ni...

[1446030 rows x 2 columns]
         tweetcreatedts                                               te

In [ ]:
for p in range(len(X_test)-4):
    X_test[p].to_csv("data\\clean\\clean_data"+str(p)+".csv", header='true', encoding='utf-8')